## Setup the Simulator

In [ ]:
%load_ext autoreload
%autoreload 2


import os
import json
import numpy as np
import sys
sys.path.append('simulation')
from unity_simulator.comm_unity import UnityCommunication
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts
from GraphReader import GraphReader, base_graph_file, init_graph_file

## Add necessary objects

In [ ]:
base_graph = GraphReader(base_graph_file)
ref_nodes = base_graph.usable_nodes_by_room
print('List of available locations :')
for room,nodelist in ref_nodes.items():
    print('##### ',room,' #####')
    furniture = list(nodelist.keys())
    furniture.sort()
    print(furniture)


Fill in objects placements in `AddObjects.py` with statements of the form :
 ```
 base_graph.add("food_apple", "ON", ref_nodes['dining_room']['kitchen_counter'])
 ```

In [ ]:
%run -i 'AddObjects.py'

In [ ]:
base_graph.write(init_graph_file)

## Write the Program

In [ ]:
from ProgramExecutor import ProgramExecutor

### Write the desired program in `program.txt`. The program should include a list of actions and comments specifying how much time you expect it to take
Example
```
## 1-2 mins
[Walk] <dining_room>
[Walk] <food_apple>
## 3-4 mins
[Grab] <food_apple>
[Walk] <kitchen_counter>
[PutBack] <food_apple> <kitchen_counter>
```

Available Actions : 

`WALK, FIND, SIT, STANDUP, GRAB, OPEN, CLOSE, PUTBACK, PUTIN, SWITCHON, SWITCHOFF, DRINK, LOOKAT, TURNTO, WIPE, RUN, PUTON, PUTOFF, GREET, DROP, READ, POINTAT, TOUCH, LIE, PUTOBJBACK, POUR, TYPE, WATCH, PUSH, PULL, MOVE, RINSE, WASH, SCRUB, SQUEEZE, PLUGIN, PLUGOUT, CUT, EAT, SLEEP, WAKEUP, RELEASE`

In [ ]:
init_graph = GraphReader(graph_file=init_graph_file)
print('Objects available for use in scripts :')
available_obj = list(init_graph.node_map.keys())
available_obj.sort()
print(available_obj)

In [ ]:
action_headers, graphs = ProgramExecutor(program_file='program.txt', graph_file=init_graph_file, node_map=init_graph.node_map).execute()

## If everything looks good, move the script to your database :)

Activity options : 

getting_out_of_bed, brushing_teeth, showering, getting_dressed, leave_home, going_to_the_bathroom, breakfast, lunch, dinner, reading, computer_work, watching_tv, hand_wash_clothes, laundry, kitchen_cleaning, vaccuum_cleaning, wash_dishes, take_out_trash, taking_medication, sleeping, cleaning, listening_to_music, socializing, diary_logging, playing_music, coming_back_home

In [ ]:
activity = input('Which activity were you doing?')
directory = os.path.join('sourcedScripts',activity)
if not os.path.exists(directory):
    raise KeyError('Could not find the activity. Please try again.')

existing_files = [int(os.path.splitext(file)[0]) for file in os.listdir(directory)]
if existing_files:
    new_file = os.path.join(directory, str(max(existing_files)+1)+'.txt')
else:
    new_file = os.path.join(directory, '0.txt')
print(f'Saving your awesome script at {new_file}')
os.rename('program.txt',new_file)
open('program.txt', 'a').close()